<br>
<br>

### 0. Create connection & SQLite3 Cursor (a method of the connection object)

In [1]:
import pandas as pd
import sqlite3 # built-in library (Python 2.x & 3.x)

dbpath = "tour_info.db" 

conn = sqlite3.connect(dbpath)
cur = conn.cursor() 

# conn.cursor(), conn.commit/rollback(), conn.close() <- conn.commit() : 저장 vs conn.rollback() : 마지막 commit 이후의 변경사항 모두 취소 
# cur.execute/executescript/executemany(), cur.fetchone/fetchall()

<br>
<br>

### 1. 여행 정보를 담을 DB TABLE 만들기 (tour_crawl)

In [2]:
script = """
DROP TABLE IF EXISTS tour_crawl;

CREATE TABLE tour_crawl(
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  title TEXT,
  price TEXT
);
""" 
cur.executescript(script) 

script = """
SELECT * FROM tour_crawl;
""" 

df = pd.read_sql_query(script, conn)
df.head()

,id,title,price


<br>
<br>

### 2. Selenium을 활용해 인터파크 투어 홈페이지에서 여행지를 입력하고 이동

- !pip install selenum == 3.11.0 << selenum 설치하는 것

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

In [4]:
driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe') 
driver.get('http://tour.interpark.com')

driver.implicitly_wait(5)

driver.find_element_by_id('SearchGNBText').send_keys('보라카이')
driver.find_element_by_class_name('search-btn').click()

<br>
<br>

### 4. 여행 상품 목록이 담긴 페이지를 지정한 수 만큼 넘기며 크롤링 & DB 저장

In [5]:
driver.find_element_by_id('li_R').click()

In [6]:
# maxpage = int(driver.find_element_by_id("totalAllCnt").text) // 10 + 1  >> 전체 page를 나눈다음 +1 을 해서 전체 페이지를 출력해줄 것임
maxpage = 3

for target_page in range(1, maxpage + 1):
    
    driver.execute_script("searchModule.SetCategoryList({}, '')".format(target_page)) # 자바스크립 함수를 실행하는 부분 {} 안에 1을 넣으면 1 page 임 , 3 을 넣으면 3 page임
    time.sleep(3) 
    print("\n{}번째 페이지의 크롤링을 시작합니다.\n".format(target_page))
    
    boxItems = driver.find_elements_by_css_selector('.panelZone > .oTravelBox > .boxList > li') # '.' 은 class를 뜻함 , '#' 이면 ID 를 의미
    
    for li in boxItems:
        # print('상품명 :', li.find_element_by_css_selector('h5.proTit').text)
        title = li.find_element_by_css_selector('h5.proTit').text
        # print('가격 :', li.find_element_by_css_selector('.proPrice').text.split('원')[0])
        price = li.find_element_by_css_selector('.proPrice').text.replace(',','').replace('원~','')
        
        sql_query = "INSERT INTO tour_crawl(title, price) values('{}',{})".format(title, price) 
        # insert_sql = "INSERT INTO tour_crawl(title, price) values({},{})".format(title,price) 
        print('BUILT SQL :', sql_query)
        
        cur.execute(sql_query) # query를 집어넣는 것 
        conn.commit() # 집어넣은 것을 저장하는 것


1번째 페이지의 크롤링을 시작합니다.

BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 4일/5일] 샌즈vs팜비치리조트 핵심일정 + 호핑할인',641000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 4일/5일] 헤난리조트 자유일정 + 호핑투어',661000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 4일/5일] 라까멜라리조트 핵심일정 + 호핑할인',331000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 4일/5일] 실속리조트 자유일정 + 호핑투어',351000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 자유 5일] 인피니티풀 최고 5성급 헤난크리스탈샌즈',757000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[부산출발_보라카이4/5/6일] 실속리조트 라까멜라+디몰투어+마사지1시간+호핑선택가능',499000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 4일/5일] 코코잉글리쉬리조트 핵심일정 + 호핑할인',301000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 4일/5일] 가든vs라군리조트 핵심일정 + 호핑할인',561000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 항공권 5일] 팬퍼시픽항공 왕복항공권',289000 )
BUILT SQL : INSERT INTO tour_crawl(title, price) values('[보라카이 

<br>
<br>

### 5. 저장된 DB 데이터를 불러와 DataFrame으로 확인 

In [7]:
script = """
SELECT * FROM tour_crawl;
""" 

df = pd.read_sql_query(script, conn)
df.head()

,id,title,price
0,1,[보라카이 4일/5일] 샌즈vs팜비치리조트 핵심일정 + 호핑할인,641000
1,2,[보라카이 4일/5일] 헤난리조트 자유일정 + 호핑투어,661000
2,3,[보라카이 4일/5일] 라까멜라리조트 핵심일정 + 호핑할인,331000
3,4,[보라카이 4일/5일] 실속리조트 자유일정 + 호핑투어,351000
4,5,[보라카이 자유 5일] 인피니티풀 최고 5성급 헤난크리스탈샌즈,757000


In [8]:
driver.close()
driver.quit()

In [9]:
df

,id,title,price
0,1,[보라카이 4일/5일] 샌즈vs팜비치리조트 핵심일정 + 호핑할인,641000
1,2,[보라카이 4일/5일] 헤난리조트 자유일정 + 호핑투어,661000
2,3,[보라카이 4일/5일] 라까멜라리조트 핵심일정 + 호핑할인,331000
3,4,[보라카이 4일/5일] 실속리조트 자유일정 + 호핑투어,351000
4,5,[보라카이 자유 5일] 인피니티풀 최고 5성급 헤난크리스탈샌즈,757000
5,6,[부산출발_보라카이4/5/6일] 실속리조트 라까멜라+디몰투어+마사지1시간+호핑선택가능,499000
6,7,[보라카이 4일/5일] 코코잉글리쉬리조트 핵심일정 + 호핑할인,301000
7,8,[보라카이 4일/5일] 가든vs라군리조트 핵심일정 + 호핑할인,561000
8,9,[보라카이 항공권 5일] 팬퍼시픽항공 왕복항공권,289000
9,10,[보라카이 4일/5일] 아잘리아리조트 핵심일정 + 호핑할인,411000
